In [1]:
import pandas as pd
import numpy as np

In [2]:
#Excels Inputs
GH = pd.read_excel('Python-Tomatoes.xls')
GH.set_index('FID',inplace=True)

Prod = pd.read_excel('Productions.xlsx')
Prod.set_index('Y',inplace=True)

CO2 = pd.read_excel('CO2.xlsx', 'Production Shares')
CO2.set_index('Years',inplace=True)

#Key Parameteres used__________________________________________________________________________

#Trasnportation factors
km=  4 *1.22   #Km/l consumption of trucks, when there is no conditioning, 1.22 average between empty and full.
Transport = 5.62 #Tons, LOADs of the trucks
km2 = 2.9*1.22 #Consumption of Refrigerated big Trucks
Transport2 = 24 #Loads of Big Trucks
km3 =4.6 #no refrigeration big truck, empty

#Subdivision of Packagin and Storing
Share_Pack=0.8449 
Share_Cond=0.1551
Pack = 1.588*Share_Pack 
Cond = 1.588*Share_Cond 

Cost_Grid = 0.01782 #€/MJel Cost of electricity from the Grid
Local_Revenue=2.63*0.09 #local cost of tomatoes


#Transformation key Factors
TT_Distance= 51 #Km to reach Trasnformation centre in Agadir
Transf=4.749 #Coefficient for transformation, MJe/Kg

#Grid for emissions
From_Grid = 0.88 #Share of electricity provided by the national grid to the region
El_Local = 0.12 #Share of electricity produced internally the borders


#Roads Dictionaries
Local_Distances = {51:79971,54:87975,67:42136,70:33152,
                  72:34090,74:57884,79:21215,82:17580,
                  94:13925,97:20500,120:19166,128:38312,
                  129:26425,131:26360,133:42807,138:40411,
                  147:31759,159:40610,168:51691}

Packaging1_Distances = {129:26425,131:25360,133:42807,147:31759,159:40612}

Packaging2_Distances = {51:45880,54:53885,67:61786,70:20889,
                      72:10913,74:23794,79:27695,82:22852,
                      94:51186,97:65593,120:56427,128:68998,
                      129:57593,131:69059,133:80339}
#______________________________________________________________________________________________

 #ELABORATION
    
#Creation of the working DataFrame
#Creation of the two levels, Prod.index stands for the years and GH.index stands for the ID of the GH

idx = pd.MultiIndex.from_product([Prod.index,GH.index],names=['Year','GH'])

col = ['TP','TT','AREA','ID_Street','Dist','%Exp']

df = pd.DataFrame ('-', idx, col)

#Repetitions
df ['AREA'] = list(GH['Size_Ha'])*len(Prod.index)
df ['ID_Street'] = list(GH['Closest Road'])*len(Prod.index)
df ['Dist'] = list(GH['Distance From Taroundant'])*len(Prod.index)

#Excel Extracts 
A = list (Prod.index)
for x in A:
    df.loc[x,'TP'] = Prod.loc[x,'TP']
    df.loc[x,'TT'] = Prod.loc[x,'TT']
    df.loc[x,'%Exp'] = Prod.loc[x,'Exp']

#Total Area of the GHs, calculated in order to be able to divide the Total Production between the differents GHs
TotalArea = df.loc[2013,'AREA'].sum() #Doesnt change over the years

#Dictionaries for the distances of the different flows:
df ['Distance_Local'] = df ['ID_Street'].map(Local_Distances)/1000

Taroundant = df ['Dist'] < 0.598
df ['PackDistance2'] = df[Taroundant]['ID_Street'].map(Packaging2_Distances)/1000

Ait_Mellou = df['Dist'] >= 0.598
df['PackDistance1'] = df[Ait_Mellou]['ID_Street'].map(Packaging1_Distances)/1000


#Energy Consumption in transportation for each GH
df['MJe_Transport']= 38.6*df['AREA']/TotalArea/Transport/km*2 *((
    df['TP'] *(1- df['%Exp'])*df['Distance_Local']) + df['TP']*df['%Exp']*(
        df.fillna(0)['PackDistance1']+df.fillna(0)['PackDistance2']))+(
            38.6*df['AREA']/TotalArea/Transport2/km2*2*df['TP']*df['%Exp']*(
                0.2*85.7+0.33*5.543+0.46*157.686)) #from packaging to outside the border


#If you need liters of diesels used
df['Fuel Used in local'] = 1/Transport/km*2/1000*df['TP']* (1- df['%Exp'])*df['Distance_Local']*df['AREA']/TotalArea
df['Fuel Export'] = 1/Transport/km*2/1000* df['TP']*df['%Exp']*(df.fillna(0)['PackDistance1']+df.fillna(0)['PackDistance2'])

#Losses due to trasnport 

df ['FinalSold_Local'] = df['TP'] * (1-df['%Exp']) * (1-(df.fillna(0)['Distance_Local'])*0.2/df['Distance_Local'].max())
df ['FinalSold_Exp']=(df ['TP']*df ['%Exp']*(1-(df.fillna(0)['PackDistance1']+df.fillna(0)['PackDistance2'])*0.2/df['Distance_Local'].max()))
ratio = (df.loc[2013,'FinalSold_Local'].sum()+df.loc[2013,'FinalSold_Exp'].sum())/df.loc[2013,'TP'].sum()

#Packaging 

df ['Packaging']= df['AREA']*df['TP']*df['%Exp']/TotalArea * Pack * 1000 *(
    1-(df.fillna(0)['PackDistance1']+df.fillna(0)['PackDistance2'])*0.2/df['Distance_Local'].max()) #Addressed Losses due to distance

df ['Pack_Emission'] = df['AREA']*df['TP']*df['%Exp']/TotalArea * 0.091 * 1000

#Conditioning
df['Conditioning'] = df['AREA']*df['TP']*df['%Exp']/TotalArea * Cond * 1000 *(
    1-(df.fillna(0)['PackDistance1']+df.fillna(0)['PackDistance2'])*0.2/df['Distance_Local'].max()) #Addressed Losses due to distance



#Waste in Trasportation ______________________________________________

Export_Coef = 1-(df.fillna(0)['PackDistance1']+df.fillna(0)['PackDistance2'])*0.2/df['Distance_Local'].max()
Export_Real = df.loc [2014,'TP']*df.loc[2014,'%Exp']*Export_Coef
Export_Ideal = df.loc [2014,'TP'] * df.loc[2014,'%Exp']
Share_Wasted_Exp = (1-(Export_Real.loc[2014].sum()/Export_Ideal.sum()))


Local_Coef = 1-(df['Distance_Local']*0.2/df['Distance_Local'].max())
Local_Real = df.loc[2014,'TP']*(1-df.loc[2014,'%Exp'])*Local_Coef
Local_Ideal =  df.loc[2014,'TP']*(1-df.loc[2014,'%Exp'])
Share_Wasted_Local = (1-(Local_Real.loc[2014].sum()/Local_Ideal.sum()))


#____________________________________________________________________________________
#Results

index = pd.MultiIndex.from_product([Prod.index,['Transport','Conditioning','Packaging','Transformation','Tot']],
                                  names=['Year','Results'])
column = ['Energy [MJeq]','%En','MJ/Kg','Energy_Cost [€]','%','Emissions','%Emis']
Results = pd.DataFrame ('-', index, column)

#MJ equivalent 
for x in A:
    
        ALPHA = Results.loc[x,'Energy [MJeq]'].loc['Conditioning'] = df.loc[x,'Conditioning'].sum()/CO2.loc[x,'ef']
        BETHA = Results.loc[x,'Energy [MJeq]'].loc['Transport'] = df.loc[x,'MJe_Transport'].sum()#+ #2* Prod.loc[x,'TT']/Transport*TT_Distance/km 
        GAMMA = Results.loc[x,'Energy [MJeq]'].loc['Packaging'] = df.loc[x,'Packaging'].sum()/CO2.loc[x,'ef']
        AMBA = Results.loc[x,'Energy [MJeq]'].loc['Transformation'] = Prod.loc[x,'TT']*Transf*1000
        Results.loc[x,'Energy [MJeq]'].loc['Tot']= GAMMA+AMBA+BETHA+ALPHA
#MJ/kg
        B1=Results.loc[x,'MJ/Kg'].loc['Conditioning'] = ALPHA/Prod.loc[x,'TP']/Prod.loc[x,'Exp']/1000
        B2=Results.loc[x,'MJ/Kg'].loc['Transport'] = BETHA/Prod.loc[x,'TP']/1000
        B3=Results.loc[x,'MJ/Kg'].loc['Packaging'] = GAMMA/Prod.loc[x,'TP']/Prod.loc[x,'Exp']/1000
        B4=Results.loc[x,'MJ/Kg'].loc['Transformation'] = AMBA/Prod.loc[x,'TT']/1000
        Results.loc[x,'MJ/Kg'].loc['Tot'] = B1+B2+B3

#% of the total energy consumption
        A1 = Results.loc[x,'%En'].loc['Conditioning'] = round(ALPHA /(ALPHA+BETHA+GAMMA+AMBA)*100,0)
        A2 = Results.loc[x,'%En'].loc['Transport'] = round(BETHA / (ALPHA+BETHA+GAMMA+AMBA) *100,0)
        A3 = Results.loc[x,'%En'].loc['Packaging'] = round(GAMMA / (ALPHA+BETHA+GAMMA+AMBA) *100,0)
        A4 = Results.loc[x,'%En'].loc['Transformation'] = round(AMBA / (ALPHA+BETHA+GAMMA+AMBA) *100,0)
        Results.loc[x,'%En'].loc['Tot'] = A1+A2+A3+A4
        
#Emissions
        
        AA = Results.loc[x,'Emissions'].loc['Conditioning'] = Results.loc[x,'Energy [MJeq]'].loc['Conditioning']/(
            3600)*CO2.loc[x,'CO2 eq g/kWh']*From_Grid + Results.loc[x,'Energy [MJeq]'].loc['Conditioning']/(
            3600)*CO2.loc[x,'CO2 eq g/kWh']*El_Local*0.346
        
        BB = Results.loc[x,'Emissions'].loc['Packaging'] = Results.loc[x,'Energy [MJeq]'].loc['Packaging']/(
            3600)*CO2.loc[x,'CO2 eq g/kWh']*From_Grid + Results.loc[x,'Energy [MJeq]'].loc['Packaging']/(
            3600)*CO2.loc[x,'CO2 eq g/kWh']*El_Local*0.346
        #df.loc[x,'Pack_Emission'].sum()
        CC = Results.loc[x,'Emissions'].loc['Transport']= BETHA*0.267/3.6
        DD = Results.loc[x,'Emissions'].loc['Transformation'] = Results.loc[x,'Energy [MJeq]'].loc['Transformation']/(
            3600)*CO2.loc[x,'CO2 eq g/kWh']*From_Grid + Results.loc[x,'Energy [MJeq]'].loc['Transformation']/(
            3600)*CO2.loc[x,'CO2 eq g/kWh']*El_Local*0.346
        dd = Results.loc[x,'Emissions'].loc['Tot']= AA+BB+CC+DD
        
        A11 = Results.loc[x,'%Emis'].loc['Conditioning'] = round( AA /(AA+BB+CC+DD)*100,0) 
        A22 = Results.loc[x,'%Emis'].loc['Transport'] = round(CC/(AA+BB+CC+DD)*100,0)
        A33 = Results.loc[x,'%Emis'].loc['Packaging'] = round(BB /(AA+BB+CC+DD)*100,0) 
        A44 = Results.loc[x,'%Emis'].loc['Transformation'] = round(DD/(AA+BB+CC+DD)*100,0) 
        Results.loc[x,'%Emis'].loc['Tot'] = A11+A22+A33+A44
        Results.loc[x,'Emissions'].loc['Trasnformation']= AMBA 
        
        

#Costs

        ALPHA2 = Results.loc[x,'Energy_Cost [€]'].loc['Conditioning'] = ALPHA*Cost_Grid*CO2.loc[x,'ef']
        BETHA2 = Results.loc[x,'Energy_Cost [€]'].loc['Transport'] = (df.loc[x,'Fuel Used in local'].sum()+
                                                                      df.loc[x,'Fuel Export'].sum())*0.9
        GAMMA2 = Results.loc[x,'Energy_Cost [€]'].loc['Packaging'] = GAMMA*Cost_Grid*CO2.loc[x,'ef']
        AMBA2 = Results.loc[x,'Energy_Cost [€]'].loc['Transformation'] = AMBA*Cost_Grid*CO2.loc[x,'ef']
        DAJE = Results.loc[x,'Energy_Cost [€]'].loc['Tot']=Prod.loc[x,'TP']*Local_Revenue*1000
                

        Results.loc[x,'%'].loc['Conditioning']=ALPHA2/DAJE*100
        Results.loc[x,'%'].loc['Transport']=BETHA2/DAJE*100
        Results.loc[x,'%'].loc['Packaging']=GAMMA2/DAJE*100
        Results.loc[x,'%'].loc['Transformation']=AMBA2/DAJE*100

Results



Energy [MJeq]  %En     MJ/Kg Energy_Cost [€]         %  \
Year Results                                                                 
2013 Transport        1.14981e+08    8  0.139675     3.19441e+06   1.63941   
     Conditioning     1.97461e+08   14  0.478951     1.66113e+06  0.852513   
     Packaging        1.07566e+09   74   2.60906     9.04895e+06   4.64403   
     Transformation   7.29446e+07    5     4.749          613645   0.31493   
     Tot              1.46104e+09  101   3.22769     1.94851e+08         -   
2014 Transport        1.13915e+08    7  0.149299     3.59934e+06   1.99297   
     Conditioning     2.29449e+08   13  0.493878      1.8719e+06   1.03648   
     Packaging        1.24991e+09   73   2.69038     1.01971e+07   5.64615   
     Transformation   1.10519e+08    6     4.749          901636  0.499239   
     Tot               1.7038e+09   99   3.33356     1.80602e+08         -   
2015 Transport        1.21718e+08    8  0.143705     3.58354e+06   1.78744   
     Conditioning     2.20581e+08   14   0.47691     1.86358e+06  0.929534   
     Packaging         1.2016e+09   75   2.59794     1.01517e+07    5.0636   
     Transformation   6.25443e+07    4     4.749          528405  0.263563   
     Tot              1.60645e+09  101   3.21856     2.00485e+08         -   
2016 Transport        1.33476e+08    7  0.151162     4.30848e+06   2.06141   
     Conditioning     2.65222e+08   14   0.47691     2.24073e+06   1.07209   
     Packaging        1.44479e+09   76   2.59794     1.22063e+07   5.84015   
     Transformation   5.06766e+07    3     4.749          428140  0.204846   
     Tot              1.89416e+09  100   3.22602     2.09006e+08         -   
2020 Transport        1.49544e+08    8  0.151735       4.858e+06   2.08246   
     Conditioning     2.72996e+08   14  0.435357     2.52653e+06   1.08304   
     Packaging        1.48713e+09   76   2.37159     1.37632e+07   5.89981   
     Transformation   5.06766e+07    3     4.749          469004  0.201047   
     Tot              1.96035e+09  101   2.95868     2.33281e+08         -   
2025 Transport        1.77252e+08    8  0.156764     6.06783e+06   2.26721   
     Conditioning     3.07504e+08   14  0.392599     3.15585e+06   1.17917   
     Packaging        1.67511e+09   76   2.13867     1.71914e+07   6.42345   
     Transformation   5.06766e+07    2     4.749          520084  0.194326   
     Tot              2.21055e+09  100   2.68803     2.67634e+08         -   
2030 Transport        2.06419e+08    8  0.161793     7.40458e+06   2.45195   
     Conditioning     3.64791e+08   14  0.381647     3.85121e+06   1.27529   
     Packaging        1.98718e+09   76     2.079     2.09793e+07   6.94708   
     Transformation   5.06766e+07    2     4.749          535009  0.177163   
     Tot              2.60907e+09  100   2.62244     3.01987e+08         -   

                       Emissions %Emis  
Year Results                            
2013 Transport       8.52773e+06     4  
     Conditioning    3.04128e+07    14  
     Packaging       1.65672e+08    77  
     Transformation  1.12349e+07     5  
     Tot             2.15848e+08   100  
2014 Transport       8.44872e+06     3  
     Conditioning    3.74086e+07    14  
     Packaging       2.03781e+08    76  
     Transformation  1.80186e+07     7  
     Tot             2.67657e+08   100  
2015 Transport        9.0274e+06     4  
     Conditioning    3.50416e+07    14  
     Packaging       1.90887e+08    78  
     Transformation  9.93581e+06     4  
     Tot             2.44892e+08   100  
2016 Transport       9.89949e+06     3  
     Conditioning    4.21333e+07    15  
     Packaging       2.29519e+08    79  
     Transformation  8.05049e+06     3  
     Tot             2.89602e+08   100  
2020 Transport       1.10912e+07     4  
     Conditioning    3.92351e+07    14  
     Packaging       2.13732e+08    79  
     Transformation  7.28328e+06     3  
     Tot             2.71341e+08   100  
2025 Transport       1.314

In [9]:
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer,'Sheet1')
Results.to_excel(writer,'Sheet2')
writer.save()

In [3]:
df

TP     TT       AREA  ID_Street      Dist      %Exp  \
Year GH                                                               
2013 0      823200  15360   0.865278         82  0.203913  0.500823   
     1      823200  15360   0.866294         82  0.205093  0.500823   
     2      823200  15360   1.186140         82  0.205520  0.500823   
     3      823200  15360   1.518515         79  0.203606  0.500823   
     4      823200  15360   0.572011         79  0.210431  0.500823   
     5      823200  15360   0.674072         79  0.204725  0.500823   
     6      823200  15360   0.834735         97  0.591509  0.500823   
     7      823200  15360   2.428210         97  0.591346  0.500823   
     8      823200  15360   5.333201         97  0.584021  0.500823   
     9      823200  15360   2.148791         97  0.576331  0.500823   
     10     823200  15360   1.713736         97  0.559098  0.500823   
     11     823200  15360   1.092483         97  0.558139  0.500823   
     12     823200  15360  11.662939         97  0.560955  0.500823   
     13     823200  15360   0.289344         97  0.558459  0.500823   
     14     823200  15360   0.519044         97  0.559130  0.500823   
     15     823200  15360   0.505751         97  0.559879  0.500823   
     16     823200  15360   1.050861         97  0.550230  0.500823   
     17     823200  15360   0.808522         97  0.550259  0.500823   
     18     823200  15360   0.837886         97  0.526392  0.500823   
     19     823200  15360   0.627533         97  0.527316  0.500823   
     20     823200  15360   0.387329         97  0.527126  0.500823   
     21     823200  15360  10.677189         97  0.516387  0.500823   
     22     823200  15360   1.024457         97  0.511158  0.500823   
     23     823200  15360   7.515170         97  0.504201  0.500823   
     24     823200  15360   3.052302         97  0.508967  0.500823   
     25     823200  15360   0.565774         97  0.511391  0.500823   
     26     823200  15360   0.587585         97  0.510341  0.500823   
     27     823200  15360  10.940078         97  0.506288  0.500823   
     28     823200  15360   7.323611         94  0.477260  0.500823   
     29     823200  15360   9.135197         97  0.576437  0.500823   
...            ...    ...        ...        ...       ...       ...   
2030 3281  1275822  10671   1.758503         51  0.310469   0.74919   
     3282  1275822  10671   1.004778         51  0.356938   0.74919   
     3283  1275822  10671   1.119348         51  0.359000   0.74919   
     3284  1275822  10671   1.219822         51  0.355719   0.74919   
     3285  1275822  10671   0.770223         51  0.359526   0.74919   
     3286  1275822  10671   0.878163         51  0.357700   0.74919   
     3287  1275822  10671   0.854279         51  0.358216   0.74919   
     3288  1275822  10671   1.173536         51  0.355397   0.74919   
     3289  1275822  10671   0.927635         51  0.357269   0.74919   
     3290  1275822  10671   1.206264         51  0.358561   0.74919   
     3291  1275822  10671   0.978284         51  0.359046   0.74919   
     3292  1275822  10671   0.873843         51  0.364228   0.74919   
     3293  1275822  10671   1.130341         51  0.364377   0.74919   
     3294  1275822  10671   1.276984         94  0.404606   0.74919   
     3295  1275822  10671   1.013730         10  4.873744   0.74919   
     3296  1275822  10671   0.573678        133  0.676330   0.74919   
     3297  1275822  10671   0.715633        133  0.676968   0.74919   
     3298  1275822  10671   0.718596        133  0.678168   0.74919   
     3299  1275822  10671   1.259594        133  0.674985   0.74919   
     3300  1275822  10671   0.569063        133  0.677837   0.74919   
     3301  1275822  10671   1.235113        133  0.675629   0.74919   
     3302  1275822  10671   0.459898        133  0.676876   0.74919   
     3303  1275822  10671   0.608201        133  0.677303   0.74919   
     3304  1275822  10671   0.976391    

In [3]:
df

TP     TT       AREA  ID_Street      Dist      %Exp  \
Year GH                                                               
2009 0      709331  17690   0.865278         82  0.203913  0.568513   
     1      709331  17690   0.866294         82  0.205093  0.568513   
     2      709331  17690   1.186140         82  0.205520  0.568513   
     3      709331  17690   1.518515         79  0.203606  0.568513   
     4      709331  17690   0.572011         79  0.210431  0.568513   
     5      709331  17690   0.674072         79  0.204725  0.568513   
     6      709331  17690   0.834735         97  0.591509  0.568513   
     7      709331  17690   2.428210         97  0.591346  0.568513   
     8      709331  17690   5.333201         97  0.584021  0.568513   
     9      709331  17690   2.148791         97  0.576331  0.568513   
     10     709331  17690   1.713736         97  0.559098  0.568513   
     11     709331  17690   1.092483         97  0.558139  0.568513   
     12     709331  17690  11.662939         97  0.560955  0.568513   
     13     709331  17690   0.289344         97  0.558459  0.568513   
     14     709331  17690   0.519044         97  0.559130  0.568513   
     15     709331  17690   0.505751         97  0.559879  0.568513   
     16     709331  17690   1.050861         97  0.550230  0.568513   
     17     709331  17690   0.808522         97  0.550259  0.568513   
     18     709331  17690   0.837886         97  0.526392  0.568513   
     19     709331  17690   0.627533         97  0.527316  0.568513   
     20     709331  17690   0.387329         97  0.527126  0.568513   
     21     709331  17690  10.677189         97  0.516387  0.568513   
     22     709331  17690   1.024457         97  0.511158  0.568513   
     23     709331  17690   7.515170         97  0.504201  0.568513   
     24     709331  17690   3.052302         97  0.508967  0.568513   
     25     709331  17690   0.565774         97  0.511391  0.568513   
     26     709331  17690   0.587585         97  0.510341  0.568513   
     27     709331  17690  10.940078         97  0.506288  0.568513   
     28     709331  17690   7.323611         94  0.477260  0.568513   
     29     709331  17690   9.135197         97  0.576437  0.568513   
...            ...    ...        ...        ...       ...       ...   
2030 3281  1275822  10671   1.758503         51  0.310469   0.74919   
     3282  1275822  10671   1.004778         51  0.356938   0.74919   
     3283  1275822  10671   1.119348         51  0.359000   0.74919   
     3284  1275822  10671   1.219822         51  0.355719   0.74919   
     3285  1275822  10671   0.770223         51  0.359526   0.74919   
     3286  1275822  10671   0.878163         51  0.357700   0.74919   
     3287  1275822  10671   0.854279         51  0.358216   0.74919   
     3288  1275822  10671   1.173536         51  0.355397   0.74919   
     3289  1275822  10671   0.927635         51  0.357269   0.74919   
     3290  1275822  10671   1.206264         51  0.358561   0.74919   
     3291  1275822  10671   0.978284         51  0.359046   0.74919   
     3292  1275822  10671   0.873843         51  0.364228   0.74919   
     3293  1275822  10671   1.130341         51  0.364377   0.74919   
     3294  1275822  10671   1.276984         94  0.404606   0.74919   
     3295  1275822  10671   1.013730         10  4.873744   0.74919   
     3296  1275822  10671   0.573678        133  0.676330   0.74919   
     3297  1275822  10671   0.715633        133  0.676968   0.74919   
     3298  1275822  10671   0.718596        133  0.678168   0.74919   
     3299  1275822  10671   1.259594        133  0.674985   0.74919   
     3300  1275822  10671   0.569063        133  0.677837   0.74919   
     3301  1275822  10671   1.235113        133  0.675629   0.74919   
     3302  1275822  10671   0.459898        133  0.676876   0.74919   
     3303  1275822  10671   0.608201        133  0.677303   0.74919   
     3304  1275822  10671   0.976391    